Loading in Packages and data path

In [32]:
# Function to Extract features from the images
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

import pandas as pd
import numpy as np
from tqdm import tqdm
from os import listdir
import shutil

train_data = 'Covid19-dataset/train'
test_data = 'Covid19-dataset/test'

## Loading in image data and creating usable model features

In [1]:
def image_feature(direc, type):
    model = InceptionV3(weights='imagenet', include_top=False)
    features = [];
    img_name = [];
    for i in listdir(direc):
        fname=direc+'/'+i
        img=image.load_img(fname,target_size=(224,224))
        x = img_to_array(img)
        x=np.expand_dims(x,axis=0)
        x=preprocess_input(x)
        feat=model.predict(x)
        feat=feat.flatten()
        features.append(feat)
        img_name.append(i+type)
    return features,img_name

# append train
t1img_features,t1img_name=image_feature('Covid19-dataset/train/Covid', "C")
t2img_features,t2img_name=image_feature('Covid19-dataset/train/Normal', "N")
t3img_features,t3img_name=image_feature('Covid19-dataset/train/Viral Pneumonia', "VP")
#trimg_features = t1img_features + t2img_features +  t3img_features
#trimg_name = t1img_name + t2img_name +  t3img_name

#append test
t1aimg_features,t1aimg_name=image_feature('Covid19-dataset/test/Covid', "C")
t2aimg_features,t2aimg_name=image_feature('Covid19-dataset/test/Normal',"N")
t3aimg_features,t3aimg_name=image_feature('Covid19-dataset/test/Viral Pneumonia',"VP")
#teimg_features = t1img_features + t2img_features +  t3img_features
#teimg_name = t1img_name + t2img_name +  t3img_name

trimg_features = t1img_features + t1aimg_features + t2img_features +t2aimg_features +  t3img_features + t3aimg_features
trimg_name = t1img_name + t1aimg_name+ t2img_name + t2aimg_name + t3img_name + t3aimg_name


2023-03-26 22:16:50.385006: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 22:17:19.060868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 170ms/step


In [2]:
#Creating Clusters
k = 3 # normal, covid, and pneumonia
clusters = KMeans(k, random_state = 40)
clusters.fit(trimg_features)
image_cluster = pd.DataFrame(trimg_name,columns=['image'])
image_cluster["clusterid"] = clusters.labels_
image_cluster


/Users/noahmcintire/Desktop/MachineLearning/venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,image,clusterid
0,COVID-00006.jpgC,0
1,COVID-00012.jpgC,0
2,06.jpegC,0
3,060.jpegC,0
4,COVID-00007.jpgC,0
...,...,...
312,0105.jpegVP,2
313,0109.jpegVP,2
314,0108.jpegVP,2
315,0104.jpegVP,2


## The following code is for creating a confusion matrix, comparing to correct values.

In [3]:
covid_cnt = 0
covid_wr1 = 0
covid_wr2 = 0
for i in image_cluster["clusterid"][0:137]:
    if i == 0:
        covid_cnt += 1
    if i == 1:
        covid_wr1 += 1
    if i == 2:
        covid_wr2 += 1

print(covid_cnt)
print(covid_wr1)
print(covid_wr2)

100
35
2


In [4]:
image_cluster["clusterid"][137:227]
normal_cnt = 0
normal_wr0 = 0
normal_wr2 = 0
for i in image_cluster["clusterid"][137:227]:
    if i == 0:
        normal_wr0 += 1
    if i == 1:
        normal_cnt += 1
    if i == 2:
        normal_wr2 += 1

print(normal_cnt)
print(normal_wr0)
print(normal_wr2)

54
0
36


In [ ]:
image_cluster["clusterid"][181:251]
vp_wr0 = 0
vp_wr1 = 0
vp_cnt = 0
for i in image_cluster["clusterid"][227:318]:
    if i == 0:
        vp_wr0 +=1
    if i == 1:
        vp_wr1 += 1
    if i == 2:
        vp_cnt += 1

print(vp_wr0)
print(vp_wr1)
print(vp_cnt)

## The below cells are additional model evaluation metrics

In [6]:
a = [0] * 137
b = [1] * 90
c = [2] * 90
true_labels = a + b + c
ari_kmeans = adjusted_rand_score(true_labels, clusters.labels_)


In [7]:
ari_kmeans


0.4636707138724491

In [8]:
from sklearn import metrics
metrics.silhouette_score(trimg_features,clusters.labels_)

0.0597677

In [9]:
metrics.calinski_harabasz_score(trimg_features, clusters.labels_)

22.693956555783227

In [10]:
metrics.davies_bouldin_score(trimg_features, clusters.labels_)

3.7057850146898885